**Comparing the different type of Wavelets at different level**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pywt

# Load the image
image = plt.imread('/content/iriss.jpg')
image_grayscale = np.mean(image, axis=2)

# Function to calculate L2 norm of detail coefficients
def calculate_l2_norm(coeffs):
    LH, HL, HH = coeffs
    x = np.add(LH, HL, HH)
    return np.linalg.norm(x, ord=2)

# Create a DataFrame to store coefficients and wavelet names
coeffs_data = []

# Function to process coefficients for a given wavelet
def process_coefficients(image, wavelet_name):
    coeffs = pywt.wavedec2(image, wavelet_name, level=3)
    LL, (LH1, HL1, HH1), (LH2, HL2, HH2), (LH3, HL3, HH3) = coeffs
    l2_norm_1 = calculate_l2_norm((LH1, HL1, HH1))
    l2_norm_2 = calculate_l2_norm((LH2, HL2, HH2))
    l2_norm_3 = calculate_l2_norm((LH3, HL3, HH3))
    coeffs_data.append({'Wavelet': wavelet_name,
                        'L2 Norm Level 1': l2_norm_1,
                        'L2 Norm Level 2': l2_norm_2,
                        'L2 Norm Level 3': l2_norm_3})

# Process coefficients for different wavelets
wavelets = ['db2', 'haar', 'sym3', 'coif1', 'rbio2.2']
for wavelet in wavelets:
    process_coefficients(image_grayscale, wavelet)

# Create DataFrame
coeffs_df = pd.DataFrame(coeffs_data)

# Display the DataFrame
print(coeffs_df)


   Wavelet  L2 Norm Level 1  L2 Norm Level 2  L2 Norm Level 3
0      db2       852.479422       557.745329       284.046263
1     haar      1068.333342       597.852548       345.546005
2     sym3       908.573574       528.818745       270.078541
3    coif1       820.397983       509.505764       275.431577
4  rbio2.2       885.699920       575.479955       359.932644


**Combining the images for 3 levels of different Wavelets**

In [ ]:
import os

# Define input and output folders
input_folder = "/content/drive/MyDrive/Output folder"
output_folder = "/content/drive/MyDrive/Pre-processed Data"

# Define wavelet parameters
wavelet_name = 'rbio2.2'
level = 3

# Preprocess the dataset
def preprocess_and_save_dataset(input_folder, output_folder, wavelet_name, level):
    # Create output folders if they don't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over images in the input folder
    for folder_name in os.listdir(input_folder):
        folder_path = os.path.join(input_folder, folder_name)
        output_folder_path = os.path.join(output_folder, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)  # Create folder for current class
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)

            # Preprocess each image
            combined_image, l2_norm = preprocess_image(image_path, wavelet_name, level)

            # Save the preprocessed data
            output_file_path = os.path.join(output_folder_path, image_name + '.npz')
            np.savez(output_file_path, combined_image=combined_image, l2_norm=l2_norm)

# Preprocess and save the training dataset
preprocess_and_save_dataset(os.path.join(input_folder, 'train'), os.path.join(output_folder, 'train'), wavelet_name, level)

# Preprocess and save the validation dataset
preprocess_and_save_dataset(os.path.join(input_folder, 'val'), os.path.join(output_folder, 'val'), wavelet_name, level)

# Preprocess and save the test dataset
preprocess_and_save_dataset(os.path.join(input_folder, 'test'), os.path.join(output_folder, 'test'), wavelet_name, level)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install split-folders

In [ ]:
import splitfolders

# Define the input and output paths
input_folder = "/content/drive/MyDrive/Iris dataset"
output_folder = "/content/drive/MyDrive/Output folder"

# Split the dataset into 80% training, 10% validation, and 10% testing
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.8, 0.1, 0.1), group_prefix=None)


Copying files: 10040 files [43:30,  3.85 files/s]


**Pre-processing the data**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
import os

# Function to preprocess an image
def preprocess_image(image_path, wavelet_name, level):
    # Load the image
    image = plt.imread(image_path)
    if len(image.shape) == 3 and image.shape[2] == 3:
        # Convert RGB image to grayscale
        image_grayscale = np.mean(image, axis=2)
    else:
        image_grayscale = image

    # Perform a discrete wavelet transform
    coeffs = pywt.wavedec2(image_grayscale, wavelet_name, level=level)
    LH, HL, HH = coeffs[-1]  # Extract detail coefficients at the specified level

    # Combine detail coefficients
    combined_image = np.add(LH, HL, HH)

    # Compute L2 norm
    l2_norm = np.linalg.norm(combined_image, ord=2)

    return combined_image, l2_norm

# Preprocess the dataset
def preprocess_dataset(input_folder, output_folder, wavelet_name, level):
    preprocessed_data = []

    # Iterate over images in the input folder
    for folder_name in os.listdir(input_folder):
        folder_path = os.path.join(input_folder, folder_name)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)

            # Preprocess each image
            combined_image, l2_norm = preprocess_image(image_path, wavelet_name, level)
            preprocessed_data.append((combined_image, l2_norm, folder_name))  # Store preprocessed data along with label

    return preprocessed_data

# Define input and output folders
input_folder = "/content/drive/MyDrive/Output folder"
output_folder = "/content/drive/MyDrive/Pre-processed Data"

# Define wavelet parameters
wavelet_name = 'rbio2.2'
level = 3

# Preprocess the training dataset
train_preprocessed_data = preprocess_dataset(os.path.join(input_folder, 'train'), output_folder, wavelet_name, level)
np.savez(output_folder + '/train_preprocessed_data.npz', train_preprocessed_data)

# Preprocess the validation dataset
val_preprocessed_data = preprocess_dataset(os.path.join(input_folder, 'val'), output_folder, wavelet_name, level)
np.savez(output_folder + '/val_preprocessed_data.npz', val_preprocessed_data)

# Preprocess the test dataset
test_preprocessed_data = preprocess_dataset(os.path.join(input_folder, 'test'), output_folder, wavelet_name, level)
np.savez(output_folder + '/test_preprocessed_data.npz', test_preprocessed_data)